# setup

In [1]:
import re
import ast
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
from pyspark.sql import types

In [2]:
# start Spark Session
from pyspark.sql import SparkSession
app_name = "finalProject"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

# check file size

In [2]:
!wc -l data/train.txt

10517429 data/train.txt


In [3]:
!wc -l data/test.txt

6042135 data/test.txt


# experimenting with DFs

In [3]:
# train = spark.read.option('header', 'false').csv('data/train.txt', sep='\t')

In [12]:
# train.write.format('parquet').save('data/train.parquet')
# train_parquet = spark.read.parquet('data/train.parquet')

In [13]:
# !du data/train.txt
# !du data/train.parquet

2497316	data/train.txt
662221	data/train.parquet


In [7]:
# %%time
# train.count()

CPU times: user 0 ns, sys: 30 ms, total: 30 ms
Wall time: 34 s


10517430

In [6]:
# %%time
# train_parquet.count()

CPU times: user 10 ms, sys: 0 ns, total: 10 ms
Wall time: 1.36 s


10517430

In [10]:
# test = spark.read.option('header','false').csv('data/test.txt', sep='\t')
# test.write.format('parquet').save('data/test.parquet')
# test_parquet = spark.read.parquet('data/test.parquet')

In [12]:
# %%time
# test_parquet.count()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 327 ms


6042135

In [3]:
train = spark.read.parquet('data/train.parquet')

In [11]:
# test = spark.read.parquet('data/test.parquet')

In [ ]:
train.summary()

In [ ]:
train.show()

In [4]:
train.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (nullable = tru

In [4]:
# df = spark.read.option('header', 'false').csv('data/toy_train.txt', sep='\t')
# df.head()

In [5]:
# %%time
# df.count()

In [6]:
# df.select(df.columns[2]).show(5)

In [7]:
# # df.write.format('parquet').save('data/toy_train.parquet')
# df = spark.read.parquet('data/toy_train.parquet')
# df.printSchema()

In [9]:
# for c in df.columns:
#     df = df.withColumnRenamed(c, c.strip('_'))
# df.columns[:5]

In [8]:
# for c in df.columns[:14]:
#     df = df.withColumn(c, df[c].cast('float'))
# df.printSchema()

In [10]:
for c in train.columns:
    train = train.withColumnRenamed(c, c.strip('_'))
train.columns[:5]

['c0', 'c1', 'c2', 'c3', 'c4']

In [11]:
%%time
for c in train.columns[:14]:
    train = train.withColumn(c, train[c].cast('float'))
train.printSchema()

root
 |-- c0: float (nullable = true)
 |-- c1: float (nullable = true)
 |-- c2: float (nullable = true)
 |-- c3: float (nullable = true)
 |-- c4: float (nullable = true)
 |-- c5: float (nullable = true)
 |-- c6: float (nullable = true)
 |-- c7: float (nullable = true)
 |-- c8: float (nullable = true)
 |-- c9: float (nullable = true)
 |-- c10: float (nullable = true)
 |-- c11: float (nullable = true)
 |-- c12: float (nullable = true)
 |-- c13: float (nullable = true)
 |-- c14: string (nullable = true)
 |-- c15: string (nullable = true)
 |-- c16: string (nullable = true)
 |-- c17: string (nullable = true)
 |-- c18: string (nullable = true)
 |-- c19: string (nullable = true)
 |-- c20: string (nullable = true)
 |-- c21: string (nullable = true)
 |-- c22: string (nullable = true)
 |-- c23: string (nullable = true)
 |-- c24: string (nullable = true)
 |-- c25: string (nullable = true)
 |-- c26: string (nullable = true)
 |-- c27: string (nullable = true)
 |-- c28: string (nullable = true)
 |--

In [12]:
%%time
stats = train[train.columns[1:14]].describe()
means = np.array(stats[stats['summary'] == 'mean'].collect())[0][1:]

CPU times: user 10 ms, sys: 30 ms, total: 40 ms
Wall time: 28.8 s


In [22]:
stats = train[train.columns[1:14]].describe()
stats.show(vertical=True)

-RECORD 0---------------------
 summary | count              
 c1      | 5851479            
 c2      | 10517430           
 c3      | 8259890            
 c4      | 8205798            
 c5      | 10229986           
 c6      | 8113327            
 c7      | 10051997           
 c8      | 10511722           
 c9      | 10051997           
 c10     | 5851479            
 c11     | 10051997           
 c12     | 2421098            
 c13     | 8205798            
-RECORD 1---------------------
 summary | mean               
 c1      | 3.328972555485545  
 c2      | 105.8503159041705  
 c3      | 23.18154818042371  
 c4      | 7.264945225315076  
 c5      | 18855.035923998334 
 c6      | 117.62881774640663 
 c7      | 15.340354060989075 
 c8      | 12.639540695615809 
 c9      | 104.34038838252738 
 c10     | 0.6040327582137781 
 c11     | 2.580706599892539  
 c12     | 0.9472768140736145 
 c13     | 8.162416988573202  
-RECORD 2---------------------
 summary | stddev             
 c1     

In [23]:
%%time
counts = []
for c in train.columns[14:]:
    count = train.agg(F.countDistinct(c).alias('c')).collect()[0]['c']
    counts.append(count)
counts = np.array(counts)
# number of unique hash  values
print(counts)

[   1396     554 2715463  726532     290      22   12059     608       3
   65969    5319 2288894    3128      26   12805 1607063      11    5017
    2119       3 1990457      17      15  139105      94   85796]
CPU times: user 120 ms, sys: 150 ms, total: 270 ms
Wall time: 50.3 s


# Experimenting with RDDs

In [16]:
toyTrainRDD = sc.textFile('data/toy_train.txt')

In [17]:
toyTrainRDD.collect()[0]

'0\t\t1\t12\t1\t62526\t\t\t3\t\t\t\t\t3\t05db9164\t08d6d899\t9143c832\tf56b7dd5\t25c83c98\t7e0ccccf\t89a13b6b\t0b153874\t7cc72ec2\tc5fe5cb9\tfa365cf9\tae1bb660\td2420e4c\tb28479f6\tbffbd637\tbad5ee18\t07c540c4\tbbf70d82\t\t\t0429f84b\t\t32c7478e\tc0d61a5c\t\t'

In [21]:
line = toyTrainRDD.collect()[0]
line

'0\t\t1\t12\t1\t62526\t\t\t3\t\t\t\t\t3\t05db9164\t08d6d899\t9143c832\tf56b7dd5\t25c83c98\t7e0ccccf\t89a13b6b\t0b153874\t7cc72ec2\tc5fe5cb9\tfa365cf9\tae1bb660\td2420e4c\tb28479f6\tbffbd637\tbad5ee18\t07c540c4\tbbf70d82\t\t\t0429f84b\t\t32c7478e\tc0d61a5c\t\t'

In [22]:
line = line.split('\t')
line

['0',
 '',
 '1',
 '12',
 '1',
 '62526',
 '',
 '',
 '3',
 '',
 '',
 '',
 '',
 '3',
 '05db9164',
 '08d6d899',
 '9143c832',
 'f56b7dd5',
 '25c83c98',
 '7e0ccccf',
 '89a13b6b',
 '0b153874',
 '7cc72ec2',
 'c5fe5cb9',
 'fa365cf9',
 'ae1bb660',
 'd2420e4c',
 'b28479f6',
 'bffbd637',
 'bad5ee18',
 '07c540c4',
 'bbf70d82',
 '',
 '',
 '0429f84b',
 '',
 '32c7478e',
 'c0d61a5c',
 '',
 '']

In [12]:
def parse(line):
    line = line.split('\t')
    key = line[0]
    values = line[1:]
    for i,v in enumerate(values[:13]):
        v = 
    
    return key, values

In [14]:
toyTrainRDD = toyTrainRDD.map(parse)

toyTrainRDD.collect()[0]

('0',
 ('',
  '1',
  '12',
  '1',
  '62526',
  '',
  '',
  '3',
  '',
  '',
  '',
  '',
  '3',
  '05db9164',
  '08d6d899',
  '9143c832',
  'f56b7dd5',
  '25c83c98',
  '7e0ccccf',
  '89a13b6b',
  '0b153874',
  '7cc72ec2',
  'c5fe5cb9',
  'fa365cf9',
  'ae1bb660',
  'd2420e4c',
  'b28479f6',
  'bffbd637',
  'bad5ee18',
  '07c540c4',
  'bbf70d82',
  '',
  '',
  '0429f84b',
  '',
  '32c7478e',
  'c0d61a5c',
  '',
  ''))